In [0]:
# The configuration has to be changed
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .appName("MyApp") \
    .getOrCreate()

In [0]:
df = spark.read.table("fordgobike_tripdataa_2007")

In [0]:
df.show()

+----------+---------------+-----------------+------------------+--------+-------------+---------------+----------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+
|start_time|start time hour|start time minute|start time seconds|end_time|end_time hour|end_time minute|end_time seconds|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|
+----------+---------------+-----------------+------------------+--------+-------------+---------------+----------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+---------

In [0]:
from pyspark.sql.functions import concat, to_timestamp, lit, when


df = df.withColumn('hour_start_str', df['start time hour'].cast("string"))
df = df.withColumn('minute_start_str', df['start time minute'].cast("string"))
df = df.withColumn('second_start_str', df['start time seconds'].cast("string"))
df = df.withColumn('time_start_str', concat(df.hour_start_str, lit(':'), df.minute_start_str, lit(':'), df.second_start_str))
df = df.withColumn('time_start', to_timestamp(df.time_start_str, 'HH:mm:ss'))
df = df.drop('time_start_str')


df = df.withColumn('hour_end_str', df['end_time hour'].cast("string"))
df = df.withColumn('minute_end_str', df['end_time minute'].cast("string"))
df = df.withColumn('second_end_str', df['end_time seconds'].cast("string"))
df = df.withColumn('time_end_str', concat(df.hour_end_str, lit(':'), df.minute_end_str, lit(':'), df.second_end_str))
df = df.withColumn('time_end', to_timestamp(df.time_end_str, 'HH:mm:ss'))
df = df.drop('time_end_str')

In [0]:

df_new = df.select(
    "*",
    when(df.time_end > df.time_start, df.time_end).otherwise(df.time_start).alias("time_end_new"),
    when(df.time_end < df.time_start, df.time_end).otherwise(df.time_start).alias("time_start_new")
)
df_new = df_new.drop('time_start')
df_new = df_new.drop('time_end')
df_new.show()

+----------+---------------+-----------------+------------------+--------+-------------+---------------+----------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+--------------+----------------+----------------+------------+--------------+--------------+-------------------+-------------------+
|start_time|start time hour|start time minute|start time seconds|end_time|end_time hour|end_time minute|end_time seconds|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|hour_start_str|minute_start_str|second_start_str|hour_end_str|minute_end_str|second_end_str|       time_end_new|     time_start_new|
+----------+---------------+----------

In [0]:
# calculate the duration in seconds of each trip
# by assuming each minute cost 0.35 cent calculate the fee for each trip
# calculate the total distance for each bike and list the top 10


from pyspark.sql.functions import unix_timestamp

# Convert timestamp columns to Unix timestamp format
df_new = df_new.withColumn("start_unix_time", unix_timestamp("time_start_new"))
df_new = df_new.withColumn("end_unix_time", unix_timestamp("time_end_new"))

# Calculate the time difference between the two columns in seconds
df_new = df_new.withColumn("time_diff_sec", df_new["end_unix_time"] - df_new["start_unix_time"])


In [0]:
df_new.select('time_diff_sec').show()

+-------------+
|time_diff_sec|
+-------------+
|         6290|
|         7599|
|         2569|
|        18973|
|          404|
|        33974|
|        38692|
|         4334|
|         4150|
|         4239|
|         3293|
|         3178|
|         2183|
|         2171|
|         2698|
|         1544|
|         1474|
|         1397|
|         1533|
|         1216|
+-------------+
only showing top 20 rows



In [0]:
df_new = df_new.withColumn("fee", df_new["time_diff_sec"] / 60 * 0.35)

In [0]:
df_new.select('fee').show()

+------------------+
|               fee|
+------------------+
| 36.69166666666666|
|           44.3275|
|14.985833333333334|
|110.67583333333332|
|2.3566666666666665|
|198.18166666666667|
|225.70333333333332|
|25.281666666666666|
|24.208333333333332|
|           24.7275|
|19.209166666666665|
|18.538333333333334|
|12.734166666666665|
|12.664166666666665|
|15.738333333333333|
| 9.006666666666666|
| 8.598333333333333|
| 8.149166666666666|
| 8.942499999999999|
| 7.093333333333333|
+------------------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import udf
from math import radians, sin, cos, sqrt, atan2

# define a Haversine function
def haversine(lat1, lon1, lat2, lon2):
    R = 6371 # radius of the earth in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    return distance

# register the function as a UDF
haversine_udf = udf(haversine)

# create a new column with the distance
df_new = df_new.withColumn("distance", haversine_udf("start_station_latitude", "start_station_longitude", "end_station_latitude", "end_station_longitude"))
df_new.select('distance').show()
     

+------------------+
|          distance|
+------------------+
|0.9428771327523804|
|3.0697281339127307|
|               0.0|
| 1.046738269017519|
|0.6364119024330074|
|               0.0|
|               0.0|
|               0.0|
|               0.0|
|               0.0|
|0.7712460090681965|
|0.7712460090681965|
|1.5174336337530918|
|1.5174336337530918|
|1.4229245705069358|
|1.0501822761772295|
|1.0501822761772295|
|2.8562249686802215|
|2.8562249686802215|
|2.8593561400296923|
+------------------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import desc

# Group the DataFrame by the "Gender" column and calculate the average age for each group
grouped_df = df_new.groupBy("bike_id").agg({"distance": "sum"})

# Sort the resulting DataFrame by the "avg(Age)" column in descending order
sorted_df = grouped_df.sort(desc("sum(distance)"))

# Select the top 10 rows and show the resulting DataFrame
top_10_df = sorted_df.limit(10)
top_10_df.show()

+-------+-----------------+
|bike_id|    sum(distance)|
+-------+-----------------+
|     68|743.3735031938387|
|   2178|721.1879797439503|
|    256|671.9186622919016|
|    235|670.1743867445255|
|   2049|656.8345089373184|
|    441|656.6472455283631|
|   2226|647.8270177934279|
|    796|646.8632808858313|
|    190|640.2957666145566|
|   2365|639.4208717838742|
+-------+-----------------+

